In [ ]:
import pandas as pd


def main():
    pd.set_option('display.max_columns', None)
    df_pple = pd.read_csv('../csv/SDB4_A_ID.csv', encoding='UTF8')
    df_pple['AGE'] = 2022 - df_pple['BTH_YR'] + 1
    df_rlps = pd.read_csv('../csv/SDB4_I_CT_RLPS.csv', encoding='UTF8')
    df_cont = pd.read_csv('../csv/SDB4_I_CT_CONT_PVT.csv', encoding ='UTF8')
    df_mgg = pd.read_csv('../csv/SDB4_I_CT_MGG_PVT.csv', encoding ='UTF8')

    #1. 차주정보 - 외국인과 법인사업자 제외
    df_pple = df_pple[df_pple['JOIN_SN_TYP'] == 1]
    df_pple = df_pple[df_pple['GENDER'] != 0]
    df_pple = df_pple[df_pple['IS_KRN'] == 1]

    #2. 보험계약관계자정보 - 법인사업자 제외
    df_rlps = df_rlps[df_rlps['IS_INDIV'] == 1]

    #3. 보험계약관계자정보 - 보험대상자가 '본인'일 경우에만 포함
    df_rlps = df_rlps[df_rlps['CT_IN_RLTN_CD'] == 1]

    #4. 보험계약관계자정보 - 거주지역코드 '정보없음' 제외
    df_rlps = df_rlps[df_rlps['GIS_CD'] != 99]

    #5. 보험계약관계자정보 - 컬럼 추리기
    df_rlps = df_rlps[['JOIN_SN', 'SCTR_CD', 'COM_SN', 'POL_SN', 'GIS_CD']]

    #6. 보험계약정보 - 단체보험이면 제외
    df_cont = df_cont[df_cont['IS_GRP_INS'] != 1]

    #7. 보험계약정보 - 주계약상태가 '정상', '만기' 일 경우에만 포함
    df_cont = df_cont[(df_cont['INS_CT_STAT_CD'] == 2) | (df_cont['INS_CT_STAT_CD'] == 5)]

    #8. 보험계약정보 - 납입주기가 '일시납'이지만 납입기간이 1 초과하는 경우 제외
    df_cont = df_cont[(df_cont['CT_PY_CYCLE_CD'] != 1) | (df_cont['CT_PY_PD'] <= 1)]

    #9. 보험계약정보 - 부활이력이 존재하지만 실효이력이 존재하지 않는 경우 제외(논리적 오류)
    df_cont = df_cont[(df_cont['CT_EFCTV_DT'] == 999912) | (df_cont['CT_RST_DT'] != 999912)]

    #10. 보험계약정보 - 보험종류가 '기타'에 해당되면 제외
    df_cont = df_cont[df_cont['INS_GBN_CD'] != 99]

    #11. 보험계약정보 - 컬럼 추리기
    df_cont = df_cont[['POL_SN', 'INS_GBN_CD', 'INS_CT_STAT_CD', 'CT_ST_DT', 'CT_END_DT', 'CT_PY_AMT', 'CT_PY_CYCLE_CD', 'CT_PY_PD', 'CT_CHNL_CD']]

    #12. 보험담보정보 - 보험담보그룹이 '기타'에 해당되면 제외
    df_mgg = df_mgg[df_mgg['MGG_GRP_CD'] != 99]

    #13. 보험담보정보 - 컬럼 추리기
    df_mgg = df_mgg[['POL_SN', 'MGG_GRP_CD', 'MGG_CD', 'MGG_CNT', 'MGG_AMT_SUM']]

    #option. 보험계약정보 - 보험종류가 '질병 상해 어린이 치아 여행자'에 해당하는 것만 포함
    df_cont = df_cont[(df_cont['INS_GBN_CD'] == 3) | (df_cont['INS_GBN_CD'] == 4) | (df_cont['INS_GBN_CD'] == 7) | (df_cont['INS_GBN_CD'] == 8) | (df_cont['INS_GBN_CD'] == 14)]

    #14. 보험종기월 null값 제외
    df_cont = df_cont[df_cont['CT_END_DT'] != 999912]

    #15. 보험시기월/보험종기월  datatime 처리
    df_cont['CT_ST_DT'] = pd.to_datetime(df_cont['CT_ST_DT'], format='%Y%m', errors='coerce')
    df_cont['CT_END_DT'] = pd.to_datetime(df_cont['CT_END_DT'], format='%Y%m', errors='coerce')

    #16. 보장기간(연단위/일단위) 컬럼 추가
    df_cont['INSU_DAYS'] = (df_cont['CT_END_DT'] - df_cont['CT_ST_DT']).dt.days
    df_cont['INSU_YEAR'] = (df_cont['CT_END_DT'] - df_cont['CT_ST_DT']).astype('timedelta64[Y]').astype('int') + 1

    df1 = pd.merge(df_pple, df_rlps, on='JOIN_SN').drop_duplicates()
    df2 = pd.merge(df1, df_cont, on='POL_SN').drop_duplicates()
    df3 = pd.merge(df2, df_mgg, on='POL_SN').drop_duplicates()

    #00 결측값 없애기
    df3 = df3.dropna(axis=0)
    df3.to_csv('../preprocessed_5.csv', encoding='UTF8', index=False)
    print(df3.head())

    print(f'df2 row_cnt : {df2.shape[0]}') # 97095
    print(f'df3 row_cnt : {df3.shape[0]}') # 791427


if __name__ == "__main__":
    main()